In [1]:
# Importing the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from math import ceil

In [2]:
#Reading data files
trips = {}
trips['trip1_acc'] = pd.read_csv("data/1/Accelerometer.csv")
trips['trip1_lac'] = pd.read_csv("data/1/Linear_Acceleration.csv")
trips['trip1_gyr'] = pd.read_csv("data/1/Gyrometer.csv")
trips['trip1_mag'] = pd.read_csv("data/1/Magnetometer.csv")
trips['trip1_lbl'] = pd.read_csv("data/1/Truth.csv")

trips['trip2_acc'] = pd.read_csv("data/2/Accelerometer.csv")
trips['trip2_lac'] = pd.read_csv("data/2/Linear_Acceleration.csv")
trips['trip2_gyr'] = pd.read_csv("data/2/Gyrometer.csv")
trips['trip2_mag'] = pd.read_csv("data/2/Magnetometer.csv")
trips['trip2_lbl'] = pd.read_csv("data/2/Truth.csv")

trips['trip3_acc'] = pd.read_csv("data/3/Accelerometer.csv")
trips['trip3_lac'] = pd.read_csv("data/3/Linear_Acceleration.csv")
trips['trip3_gyr'] = pd.read_csv("data/3/Gyrometer.csv")
trips['trip3_mag'] = pd.read_csv("data/3/Magnetometer.csv")
trips['trip3_lbl'] = pd.read_csv("data/3/Truth.csv")

trips['trip4_acc'] = pd.read_csv("data/4/Accelerometer.csv")
trips['trip4_lac'] = pd.read_csv("data/4/Linear_Acceleration.csv")
trips['trip4_gyr'] = pd.read_csv("data/4/Gyrometer.csv")
trips['trip4_mag'] = pd.read_csv("data/4/Magnetometer.csv")
trips['trip4_lbl'] = pd.read_csv("data/4/Truth.csv")

In [3]:
trips['trip1_acc'].head(5)

,timestamp,uptimeNanos,x,y,z
0,14/05/2016 10:54:33,11537640270059,-0.161602,0.120174,9.596758
1,14/05/2016 10:54:33,11537650128140,-0.122628,0.315638,9.425655
2,14/05/2016 10:54:33,11537659894659,-0.178777,0.330180,9.445955
3,14/05/2016 10:54:33,11537679549779,0.016043,0.038759,9.528445
4,14/05/2016 10:54:33,11537699204899,0.141716,-0.162492,9.756854


In [4]:
# Irrelevant timestamp column dropping
# Also, determine smallest and largest window sizes for the events
min_window = 1000000
max_window = 0

for name,trip in trips.items():
    if 'timestamp' in trip.columns:
        trip.drop(['timestamp'], axis=1, inplace=True)
    
    if 'uptimeNanos' in trip.columns:
        uptimeNanos = np.array(trip['uptimeNanos'])
        timeInSec = (uptimeNanos - uptimeNanos[0])/1000000000
        trip['timeInSec'] = timeInSec
        trip.drop(['uptimeNanos'], axis=1, inplace=True)
        
    if 'event' in trip.columns:
        maxW = max(trip.finish - trip.start)
        minW = min(trip.finish - trip.start)
        
        if maxW > max_window:
            max_window = maxW
        if minW < min_window:
            min_window = minW

In [5]:
trips['trip1_acc'].head(5)

,x,y,z,timeInSec
0,-0.161602,0.120174,9.596758,0.000000
1,-0.122628,0.315638,9.425655,0.009858
2,-0.178777,0.330180,9.445955,0.019625
3,0.016043,0.038759,9.528445,0.039280
4,0.141716,-0.162492,9.756854,0.058935


In [6]:
# Rounding of the window size to the ceil of the time difference between start and finish of the event 
print("Minimum event duration: ", min_window, "s   Minimum Window Size: " , ceil(min_window), "s")
min_window = ceil(min_window)
print("Maximum event duration: ", max_window, "s   Maximum Window Size: " , ceil(max_window), "s")
max_window = ceil(max_window)

Minimum event duration:  1.5999999999999979 s   Minimum Window Size:  2 s
Maximum event duration:  4.899999999999977 s   Maximum Window Size:  5 s


In [7]:
# Storing the indices of 1sec frames 
allTripPerSecIndex = {}
for name,trip in trips.items():
    if 'event' not in trip.columns:
        tmpFrame = pd.DataFrame(columns=['perSecIndex'])
        count = 0
        for i in range(0,len(trip)-1,1):
            if trip['timeInSec'][i]-count > 1:
                tmpFrame = tmpFrame.append({'perSecIndex':i-1},ignore_index=True)
                count = count + 1
        allTripPerSecIndex[name+'_perSecIndex'] = tmpFrame

In [8]:
# Function getFeatureNames
# Input:
#     s = sensor_name
#     w = window_size
# Output:
#     l = [feature names as list, feature name as dictionary]

# params = [Mean, Median, Std Deviation, Mean Abs Deviation, Min, Max, Tendency]
def getFeatureNames( s, w ):
    l = []
    c = []
    d = {}
    params = ['mean','median','sd','td']
    
    for i in range(1,w+1):
        for j in range(0,len(params),1):
            v1 = s + '_' + params[j] + '_x' + str(i)
            v2 = s + '_' + params[j] + '_y' + str(i)
            v3 = s + '_' + params[j] + '_z' + str(i)
            
            if params[j] != 'td':
                c.append(v1)
                c.append(v2)
                c.append(v3)
                
                d[v1] = 0.0
                d[v2] = 0.0
                d[v3] = 0.0
            else:
                if i != w:                    
                    c.append(v1)
                    c.append(v2)
                    c.append(v3)
                    
                    d[v1] = 0.0
                    d[v2] = 0.0
                    d[v3] = 0.0
    l.append(c)
    l.append(d)
    
    return l

In [ ]:
# Considering window_size = 3
w = 3
dataSensorwise = {}
for name,tripIndex in allTripPerSecIndex.items():
    i = 0
    j = w
    end = len(tripIndex)
    l = getFeatureNames(name[6:len(name)-12],w)
    tripFeature = pd.DataFrame(columns=l[0])
    tripRawData = trips[name[:len(name)-12]]
    
    featureVector = l[1]   
    key_mean = name[6:len(name)-12] + '_mean_'
    key_median = name[6:len(name)-12] + '_median_'
    key_sd = name[6:len(name)-12] + '_sd_'
    key_td = name[6:len(name)-12] + '_td_'
    setLastInd = 0
    
    while(j<=end):    
        lastInd = setLastInd
        frameCount = 1
        
        for k in range(i,j,1):            
            ind = tripIndex.perSecIndex.values[k]
            
            featureVector[key_mean + 'x' + str(frameCount)] = tripRawData.x[lastInd:ind+1].mean()
            featureVector[key_mean + 'y' + str(frameCount)] = tripRawData.y[lastInd:ind+1].mean()
            featureVector[key_mean + 'z' + str(frameCount)] = tripRawData.z[lastInd:ind+1].mean()
            
            featureVector[key_median + 'x' + str(frameCount)] = tripRawData.x[lastInd:ind+1].median()
            featureVector[key_median + 'y' + str(frameCount)] = tripRawData.y[lastInd:ind+1].median()
            featureVector[key_median + 'z' + str(frameCount)] = tripRawData.z[lastInd:ind+1].median()
            
            featureVector[key_sd + 'x' + str(frameCount)] = tripRawData.x[lastInd:ind+1].std()
            featureVector[key_sd + 'y' + str(frameCount)] = tripRawData.y[lastInd:ind+1].std()
            featureVector[key_sd + 'z' + str(frameCount)] = tripRawData.z[lastInd:ind+1].std()                       
            
            # Set td only if it is not first frame
            if frameCount != 1:
                featureVector[key_td + 'x' + str(frameCount-1)] = featureVector[key_mean + 'x' + str(frameCount)]/featureVector[key_mean + 'x' + str(frameCount-1)]
                featureVector[key_td + 'y' + str(frameCount-1)] = featureVector[key_mean + 'y' + str(frameCount)]/featureVector[key_mean + 'y' + str(frameCount-1)]
                featureVector[key_td + 'z' + str(frameCount-1)] = featureVector[key_mean + 'z' + str(frameCount)]/featureVector[key_mean + 'z' + str(frameCount-1)]  
            else:
                setLastInd = ind;
                
            frameCount = frameCount + 1
        
        tripFeature = tripFeature.append(featureVector,ignore_index=True) 
        i = i + 1
        j = j + 1
    dataSensorwise[name[:len(name)-12]] = tripFeature       
            

In [ ]:
# Merging all sensor attribute vectors into a common attribute vector
dataConsolidated = {}
dataConsolidated['trip1'] = pd.DataFrame()
dataConsolidated['trip2'] = pd.DataFrame()
dataConsolidated['trip3'] = pd.DataFrame()
dataConsolidated['trip4'] = pd.DataFrame()

for name, vec in dataSensorwise.items():
    if name[:5] == 'trip1':
        dataConsolidated['trip1'] = pd.concat([dataConsolidated['trip1'],vec],axis=1)
    elif name[:5] == 'trip2':
        dataConsolidated['trip2'] = pd.concat([dataConsolidated['trip2'],vec],axis=1)
    elif name[:5] == 'trip3':
        dataConsolidated['trip3'] = pd.concat([dataConsolidated['trip3'],vec],axis=1)
    else:
        dataConsolidated['trip4'] = pd.concat([dataConsolidated['trip4'],vec],axis=1)

In [ ]:
# Appending the column with key in label frame
for name, trip in trips.items():
    if name[6:len(name)] == 'lbl':
        trip['key'] = 0

In [ ]:
# Appending the column with key in data frame
for name,tripDataConsolidated in dataConsolidated.items():
    tripDataConsolidated['key'] = 0
    tripDataConsolidated['startTime'] = np.arange(start=0,step=1, stop=len(tripDataConsolidated))
    tripDataConsolidated['endTime'] = np.arange(start=3,step=1, stop=len(tripDataConsolidated)+3)
    dataConsolidated[name] = pd.merge(tripDataConsolidated,trips[name+'_lbl'],on='key',how='inner')  
    dataConsolidated[name].drop(dataConsolidated[name][(dataConsolidated[name].start<dataConsolidated[name].startTime) |
                                 (dataConsolidated[name].start>=dataConsolidated[name].endTime)].index,inplace=True)
    dataConsolidated[name].drop(['key','start','finish','startTime','endTime'],axis=1,inplace=True)
    dataConsolidated[name].reset_index(drop=True, inplace=True)

In [ ]:
dataConsolidated['trip1']

In [ ]:
# Preparing the final data frame, that will be utilized for ML models
dataset = pd.DataFrame()
for name, trip in dataConsolidated.items():
    dataset = pd.concat([dataset, dataConsolidated[name]], axis=0)


In [ ]:
dataset = dataset.reset_index(drop=True)

In [ ]:
dataset

In [ ]:
eventDist = dataset.event.value_counts()
sizes = [eventDist[0],eventDist[4],eventDist[3],eventDist[5],eventDist[2],eventDist[1],eventDist[6]]

fig1, ax1 = plt.subplots(figsize=(6,6))
ax1.pie(sizes, labels=dataset.event.unique(), autopct='%1.1f%%', shadow=True)
ax1.axis('equal')
plt.show()

In [ ]:
plt.figure(figsize=(20, 6))
sns.set_style("whitegrid") 
sns.boxplot(x = 'event', y = 'acc_mean_x1', data = dataset).set_title('Relation between event and acc_mean_x1')